In [1]:
from utils.datasets.daquar import load_daquar_datasets

datasets_ = load_daquar_datasets()

Found cached dataset csv (/home/z1158649/.cache/huggingface/datasets/csv/default-0c1e51b1cb5a79db/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from torch.utils.data import DataLoader, SequentialSampler
import datasets
from collators.daquar import DaquarCollator
from models.backbones.configs import BEITConfig, DISTILBERTConfig
from utils.datasets.daquar import DaquarAnswerSpace
from transforms.vqa_v2 import image_augmentation_for_vqa_v2

answer_space = DaquarAnswerSpace()

image_processor = BEITConfig.get_image_processor()

tokenizer = DISTILBERTConfig.get_tokenizer()


dataloader = DataLoader(
    datasets_[datasets.Split.VALIDATION],
    sampler=SequentialSampler(datasets_[datasets.Split.VALIDATION]),
    collate_fn=DaquarCollator(
        tokenizer=tokenizer,
        image_processor=image_processor,
        image_encoder_config=BEITConfig,
        text_encoder_config=DISTILBERTConfig,
        image_transforms=image_augmentation_for_vqa_v2,
        answer_space=answer_space,
    ),
    batch_size=64,
    num_workers=6,
)

/miniconda/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: otobuf is an invalid version and will not be supported in a future release
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [3]:
batch = next(iter(dataloader))

/home/z1158649/vqa/utils/datasets/answer_space.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._answers_space = self.answers_space.append(
/home/z1158649/vqa/utils/datasets/answer_space.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._answers_space = self.answers_space.append(
/home/z1158649/vqa/utils/datasets/answer_space.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._answers_space = self.answers_space.append(
/home/z1158649/vqa/utils/datasets/answer_space.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._answers_space = self.answers_space.append(
/home/z1158649/vqa/utils/datasets/answer_space.p

In [4]:
batch

{'question': ['what is the colour of the sofas',
  'what is the largest object',
  'what is the red object below the kitchen counter on the left side of the stool',
  'what is on the table',
  'what is right of sofa',
  'what is at the right side',
  'what is behind the bed',
  'what is the object close to the wall left of the window',
  'what is on the bed',
  'what are around the table',
  'what are the things on the piano',
  'what is left side of the bed',
  'what is at the right side of the bed',
  'what is the object close to the wall behind the chair',
  'what are the colours identifiable on the whiteboard',
  'what is the object found on the table',
  'what is to left of ladder',
  'what is the colour of the bed cover',
  'what is the red and gray object in front of the mirror and close to the edge of the desk',
  'what is the colour of the lamp on the night stand',
  'how many chairs are around the table',
  'how many sinks are there',
  'how many computers are visible',
  'wh

In [ ]:
len(answer_space)